In [1]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.2.0
keras= 2.3.0-tf


In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath=PATH_DATA):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.diseases=[]
        self.ages=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            print("n=",n)
            speaker,disease,age,number=n.split("-")
            self.speakers.append(speaker)
            self.diseases.append(disease)
            self.ages.append(age)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases)
        self.iDiseaseLabels=self.leDiseases.transform(self.diseases)
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages)
        self.iAgeLabels=self.leAges.transform(self.ages)
        print(len(self.diseases))
        print(self.diseases)
        print(self.ages)
        print(self.speakers)

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNames= 673
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/AAJ-N-5-00002.wav
n= AAJ-N-5-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ACD-N-4-00005.wav
n= ACD-N-4-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/AMA-N-5-00001.wav
n= AMA-N-5-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/BHH-N-7-00001.wav
n= BHH-N-7-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/BHH-N-7-00002.wav
n= BHH-N-7-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/CAR-N-7-00001.wav
n= CAR-

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VVA-SMS-17-00010.wav
n= VVA-SMS-17-00010
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/TBD-N-5-00004.wav
n= TBD-N-5-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/TBD-N-5-00005.wav
n= TBD-N-5-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/TJI-N-4-00003.wav
n= TJI-N-4-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/2JA-SMS-8-00005.wav
n= 2JA-SMS-8-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/TVD-N-6-00001.wav
n= TVD-N-6-00001
22050 

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/LAR-N-7-00001.wav
n= LAR-N-7-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/LEY-N-5-00004.wav
n= LEY-N-5-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/LEY-N-5-00005.wav
n= LEY-N-5-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/LMA-N-7-00001.wav
n= LMA-N-7-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/LRA-N-7-000015.wav
n= LRA-N-7-000015
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/MAJ-N-7-00004.wav
n= MAJ-N-7-00004
22050 wave= (2

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/JMA-N-7-00004.wav
n= JMA-N-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/HSI-N-5-00005.wav
n= HSI-N-5-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/NVA-N-9-00003.wav
n= NVA-N-9-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/HXA-N-5-00005.wav
n= HXA-N-5-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ACR-N-7-00004.wav
n= ACR-N-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ACI-N-6-00003.wav
n= ACI-N-6-00003
22050 wave= (220

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ACD-N-4-00001.wav
n= ACD-N-4-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/DMH-N-6-00001.wav
n= DMH-N-6-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/AIE-N-6-00002.wav
n= AIE-N-6-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/EMA-N-6-00001.wav
n= EMA-N-6-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/FFR-N-6-00003.wav
n= FFR-N-6-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/JCL-N-7-00004.wav
n= JCL-N-7-00004
22050 wave= (220

...

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/EMA-N-6-00005.wav
n= EMA-N-6-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/PAJ-N-4-00005.wav
n= PAJ-N-4-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/BBR-N-7-00004.wav
n= BBR-N-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/CPD-N-5-00004.wav
n= CPD-N-5-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/FCD-N-5-00004.wav
n= FCD-N-5-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/RPC-N-10-00003.wav
n= RPC-N-10-00003
22050 wave= (2

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/FLS-N-7-00005.wav
n= FLS-N-7-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ENF-SMS-15-00004.wav
n= ENF-SMS-15-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ENF-SMS-15-00002.wav
n= ENF-SMS-15-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/TBD-N-5-00002.wav
n= TBD-N-5-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/AMA-N-5-00005.wav
n= AMA-N-5-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/PLA-N-4-00004.wav
n= PLA-N-4-00004
2205

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/CSR-N-4-00002.wav
n= CSR-N-4-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/PMD-N-7-00003.wav
n= PMD-N-7-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/FCD-N-5-00001.wav
n= FCD-N-5-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/2JA-SMS-8-00003.wav
n= 2JA-SMS-8-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/HGH-N-9-00001.wav
n= HGH-N-9-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/RDA-N-4-00005.wav
n= RDA-N-4-00005
22050 wave= 

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/PGM-N-6-00001.wav
n= PGM-N-6-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/AMA-N-5-00002.wav
n= AMA-N-5-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/FGJ-N-7-00004.wav
n= FGJ-N-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VTR-N-5-00001.wav
n= VTR-N-5-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/RMD-N-4-00002.wav
n= RMD-N-4-00002
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/ORD-N-7-00005.wav
n= ORD-N-7-00005
22050 wave= (220

22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/SSA-SMS-16-00003.wav
n= SSA-SMS-16-00003
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VJA-SMS-7-00001.wav
n= VJA-SMS-7-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VJA-SMS-7-00004.wav
n= VJA-SMS-7-00004
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VJA-SMS-7-00005.wav
n= VJA-SMS-7-00005
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VVA-SMS-17-00001.wav
n= VVA-SMS-17-00001
22050 wave= (22050,) mfcc= (44, 20) file= /home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201/VVA-SMS-17-00002.wav
n= VVA

In [3]:
trainX, trainYdiseases, trainYspeakers, trainYages = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=max(trainYdiseases)+1
outputSpeakers_dim=max(trainYspeakers)+1
outputAges_dim=max(trainYages)+1

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape)

2 134 18
(673, 30, 20) (673, 2) (673, 134) (673, 18)


In [4]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,hiden_dim=128):
    input = Input(shape=(30,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)(x)
    print(x.shape)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim)(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim)(x)
    sd= Dense(hiden_dim)(x)
    ad= Dense(hiden_dim)(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages])
    return model

In [8]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 30, 20)
(None, 28, 128)
(None, 13, 128)
(None, 6, 128)
(None, 128)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 20)]     0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 28, 128)      7808        input_2[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 28, 128)      0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 13, 128)      49280       dropout_4[0][0]                  
___________

In [9]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat], batch_size=10,epochs=5,validation_split=0.2)

Epoch 1/5


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_1/conv1d_3/conv1d (defined at <ipython-input-9-e9a18a613c83>:2) ]] [Op:__inference_train_function_3155]

Function call stack:
train_function


In [20]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 30, 20)
[[1.0566245e-17 1.6993784e-29 1.2629809e-15 8.0412259e-26 3.6855405e-29
  5.2542363e-30 1.8826095e-17 2.3430210e-21 4.9290455e-24 1.1396187e-33
  1.0000000e+00 5.5632385e-27 1.0227323e-27 1.2566632e-27 1.6294543e-26
  3.2783657e-27 8.5621296e-25]] [[1.1922505e-14 7.2233626e-17 6.6164611e-15 1.0000000e+00]]
10 = 1.0
3 = 1.0
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']


In [59]:
!sox -r 16000 -t alsa default recording.wav silence 1 0.1 1% 1 1.5 1%
!aplay "recording.wav"


Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 16000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:03.33 [00:00:00.00] Out:15.4k [      |      ]        Clip:0    
Done.
Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [24]:
!aplay "recording.wav"

Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [25]:
sample=wav2mfcc("recording.wav")
sample=np.expand_dims(np.array(sample),axis=0)
sample.shape

16000 wave= (3550,) mfcc= (7, 20) file= recording.wav


(1, 30, 20)

In [26]:
predWord,predSpeaker=model.predict([sample])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

[[9.5779401e-01 5.3499174e-05 3.0562773e-02 6.6761061e-04 1.0508731e-03
  4.9302249e-05 3.6217431e-03 2.6359034e-05 7.7112427e-06 2.2256410e-05
  5.8752643e-03 1.0338744e-04 2.8792516e-05 2.8994787e-06 9.3851040e-06
  1.8573144e-06 1.2219230e-04]] [[5.5516221e-06 7.6552706e-06 1.0617006e-05 9.9997616e-01]]
0 = 0.957794
3 = 0.99997616
paco  ha dicho ' avanza '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
